# Numpy

[Numpy](https://www.numpy.org/) is the foundation of most of what you will do with scientific python. Modules like [scipy](https://www.scipy.org/), [pandas](https://pandas.pydata.org/) and [scikit-learn](https://scikit-learn.org/stable/) are built on top of numpy. It is the _linga franca_ for numerical work in python.

In [ ]:
import numpy as np

When you are first introduced to Python, one of the big selling points is that it isn't statically typed (dynamic/strong). You can do things like

In [ ]:
a = 1
a = 'apple'
print(a)

And you won't get complaints from python about a being an integer. This is a _big_ advantage for python, and it works with collections as well

In [ ]:
myList = [1, 3., 'Ian', [range(3)], {'not': 'a good idea'}, lambda x: x**2, object ]
myList

Lists in python are about as general as they could be. This is very flexible and it lets you do some fancy things, but it has a price. The Python interpreter can't make any assumptions about what will come next in a list; everything has to be treated as a generic object. Python does a good job of hiding the complexity of doing this, but as the lists get longer and more complex the overhead gets larger, and eventually perfomance becomes unacceptable.

One solution to this is to use a statically typed language like C[<sup>1</sup>](#fn1 "footnote and tooltip 1"). There the burden of figuring out object types is left to the programmer, and the compiler can be much more efficient operating on them. A good example would be an array of `double`s. In memory, these will be allocated contiguously so when you need to jump to the 1402th double, you can do it with very simple arithmetic. Python has a much harder time because the memory allocated for your list could be a horrible mixture of all the different things you've stuffed in there. 

## The `ndarray`

Numpy attempts to keep the advantages of Python without sacrificing the speed of static typing by adding the concept of homogeneous collections to python: `ndarray`s. The `ndarray` is the foundational concept in numpy, it is an array object which represents a multidimensional, homogeneous array of fixed-size items and most commonly these items will be numbers.

In [ ]:
%%timeit
for i in range(1000000):
    i*i

In [ ]:
%timeit np.arange(1000000)**2

`ndarray`s look like python lists, but they are fundamentally different, e.g.

In [ ]:
a = [1, 2, 3, 4]
b = [5, 6, 7, 8]
a+b

In [ ]:
na = np.array([1, 2, 3, 4])
nb = np.array([5, 6, 7, 8])
na + nb

`numpy` was able to assume that the things in the `ndarray` were the compatible types and vectorize the addition, if we want the same thing with python lists, we have to jump through some hoops

In [ ]:
[ i + j for i, j in zip(a, b) ] 

_If you dig deep enough in some of the numpy/scipy code you will find that the actual instructions you are executing were compiled from fortran and C. In general you can pass things quite easily between existing libraries and python, but fortran and C use different storage orders for multi-dimensional arrays so you have to be a little bit careful_

Let's take a look at what actually makes up an `ndarray`.

In [ ]:
na = np.array([1,2,3,4,5])
na

In [ ]:
attr_names = [attr_name for attr_name in dir(np.ndarray)
              if not callable(getattr(np.ndarray, attr_name)) 
              and not attr_name[:2] == '__']

attr_help = {a : getattr(np.ndarray, a).__doc__.split('\n')[0] for a in attr_names}

In [ ]:
attr_help

Using these attributes, numpy can use some of the same tricks that statically typed languges use because the Python interpreter can now infer the memory layout.

In [ ]:
a = np.arange(10, dtype=float)
a.nbytes

These attributes allow us to look at the same object in memory in multiple ways. `numpy` calls this concept a `view` of the original `ndarray`. Wherever possible, numpy will always try to use views rather than copying data. This is important to numpy's efficiency, but you have to remember that modifications will alter all views,

In [ ]:
b = a[::2]
b.base is a

In [ ]:
c = a.reshape(5,2)
c[3,0] = -1
c.base is a

In [ ]:
a

In [ ]:
c is a

In [ ]:
d = a[[0, 3, 7]]
d.base is a

In [ ]:
a.astype(int).base is a

### Creating `ndarray`s
numpy has some convenience methods for creating new `ndarray`s. As you saw above, one way to create an ndarray is to pass a iterable with the values to `np.array`. Here are some others...

Using `np.array` directly

In [ ]:
a1 = np.array([1, 2, 3, 4, 5, 6])
a1

`np.arange` will generate numbers between limits

In [ ]:
a2 = np.arange(0, 100)
a2

In [ ]:
a22 = np.arange(0, 10, dtype=float)
a22

`np.linspace` is very useful, you tell it where to start and stop and how many samples you need and linspace does the rest. Here we will create numbers 100 numbers between 0 and 1 (inclusive), linearly spaced

In [ ]:
li1 = np.linspace(0, 1, 20)
li1

_Exercise: Use linspace to generate 25 of the values between 1 and 5, excluding the endpoint_

The `np.zeros` and `np.ones` functions will generate an `ndarray` full of those values. The first argument is the shape which you can give as an integer (for 1d arrays) or a sequence (for n-dimensional arrays). You can also pass the `dtype=` argument to tell it what type of number you are looking for.

In [ ]:
# z1 100 integer zeros
z1 = np.zeros(100, dtype=int)

# z2 a 5,5 array of float64 zeros
z2 = 5 * np.ones((5, 5), dtype='complex128')
z2.shape

The `np.eye` function generates a 2D array with ones down the diagonal, zeros elsewhere

In [ ]:
# e1 a 5x5 array with ones down the diagonal
e1 = np.eye(5, dtype=np.float64)
e1

_**Exercise**: The diag function lets you specify diagonal arrays by giving the non-zero entries down the diagonal. Try creating a 5x5 array with ones down the diagonal. Have a look at the help and see if you can do the same thing with the ones offset above and below the diagonal_

We'll discuss the `numpy.random` module in detail later on, but for now, it has some useful convenience methods. `np.random.randint` returns random integers. Take a look at the help on the method then create a name `r1` with an array of `4x5` random numbers between `0` and `10`.


In [ ]:
# r1 a 4x5 array of random integers between 0 & 10, 
np.random.randint(0, 10, size=(4, 5))

Numpy floats have representations for certain special values `np.nan`, `np.Inf` etc. These objects have well defined comparisons

In [ ]:
np.NaN == np.NaN

In [ ]:
np.NINF < 0

In [ ]:
np.NAN < np.Infinity

## Indexing and Slicing

Now that we have some `ndarray`s to play with, lets look at using them. Of course, `ndarray`s are zero indexed and for simple operations, slicing works as it normally does in Python, but since slices can be represented as a `view`, numpy doesn't copy the array when returning a slice, it returns a view. If you _really_ need a new array, the `.copy()` method will do that.

In [ ]:
# First element of a2
a2[0]

In [ ]:
# 2nd to last element of a2
a2[-2]

We can update `ndarray` in place by index


In [ ]:
a2[-1] = 0
a2[-1]

In [ ]:
# Values between 3 and 19 of a2
a2[3:19]

In [ ]:
# Every third value between 3 and 19
a2[3:19:3]

Using negative is allowed for all three parts of the slice, but for the step you have to think a bit

In [ ]:
# Values between -10 and -2
a2[-10:-2]

Notice that the first argument of the slice is still inclusive and the second is not. If we omit a value when specifying the slice `start` defaults to 0, `end` defaults to the last element and `step` defaults to 1.

In [ ]:
a2[::3]

For multi-dimensional arrays the indexing notation is similar

In [ ]:
b = np.arange(100)
b.shape = (10, 10)
b

How about row 0, column 3 (remember python is 0 indexed)

In [ ]:
b[0,3]

In [ ]:
b[:, -1]

Or the fifth column of the first two rows

_**Exercise**: Create a 2d array with 1 on the border and 0 inside_

### Fancy Indexing

Fancy Indexing is the idea of using another array of indices, it is useful when the combinations you want to select become a bit more complicated. We'll see more of this in pandas but as a few quick examples...

In [ ]:
v1=np.arange(27)[::-1]
v1

In [ ]:
v1[[1, 4, 6]]

In [ ]:
v2 = v1.reshape(3,9)
v2

In higher dimensions think of `zip`-ping together the arguements, e.g. first row (index 0), fourth column (index 3) column

In [ ]:
v2[[0, 1, 1], [3, 7, 8]]

You can also index with logical values

In [ ]:
rng = np.random.default_rng(42)
ar = rng.integers(0, 10, size=25)

# Negate any values between 3 and 8
ar[(3 < ar) & (ar < 8)] *= -1
ar

The `np.argmax` and `np.argmin` functions can return the maximum or minimum value along some axis and can be used together with fancy indexing to grab values of interest

In [ ]:
ar.shape = (5,5)
np.argmax(ar)

`argmax` returns the index for a flattened copy of the array, but the `unravel_index` function can help us translate that to the position we actually want (I'm deliberately ignoring functions like `max` here).

In [ ]:
ar[np.unravel_index(np.argmax(ar), ar.shape)]

*N.B. Fancy indexing usually creates copies of the `ndarray` because you usually can't reconstruct the selection with simple algebra*

## Reshaping Arrays

Sometimes it is convenient to reshape arrays. I did this above by setting the `.shape` attribute but numpy arrays also have a reshape method.

In [ ]:
c = np.arange(27)
c

`reshape` expects a sequence as the first argument (e.g. a tuple) so

In [ ]:
d = c.reshape((3, 9))
d

One argument to reshape can be `-1`, in which case the number is determined by the number and size of remaining dimensions.

In [ ]:
# A 256x256 array of pixels with values between 0 and 4 reshaped to 2 dimensional
w, h = 256, 256
I = np.random.randint(0, 4, (h, w, 3)).astype(np.ubyte)
I.reshape(-1, 3).shape

Reshaping isn't enough to provoke numpy to copy the data, all it needs to do is make a new view on the same data

In [ ]:
d.base is c

Another common reshaping task is to add dimension(s) to an existing array. numpy has a special `newaxis` object for this task. This is a powerful idea when combined with numpy's broadcasting rules (see below).

In [ ]:
e = np.arange(10)
# 10 x 1
e[:, np.newaxis]

In [ ]:
# 1 x 10
e[np.newaxis,:]

In [ ]:
# 1 x 1 x 10
e[np.newaxis, np.newaxis, :]

Sometimes it is necessary to go back and forth between flat (1d) and multi-dimensional views of an array. The `np.ravel`, `np.flat` functions and `ndarray.flatten` method as well as things like `np.unravel_index` can help with that.

## Stacking & Splitting ndarrays

You can combine general ndarrays with the `np.concatenate` and split them with `np.split`. There are also a number of convenience methods for commonly used shapes.

 * `_r`
 * `_c`
 * `hstack`
 * `vstack`
 * `hsplit`
 * `vsplit`

`_r` translates slice objects to concatenation along the first axis.

In [ ]:
np.r_[np.array([1,2,3]), 0, 0, np.array([4,5,6])]


`_c` translates slice objects to concatenation along the second axis.

In [ ]:
np.c_[np.array([1,2,3]), np.array([4,5,6])]

`hstack`  and `vstack` do something similar, all 4 are basically ways of calling the more generic `np.concatenate` or `np.stack` functions for specific cases

In higher dimensions `concatenate` or `stack` should do what you need, but you need to manually tell it which axis to use for the stacking.

In [ ]:
np.concatenate((np.array([1,2,3]), [0], [0], np.array([4, 5, 6])))

In [ ]:
np.concatenate((np.array([[1, 2, 3]]).T, np.array([[4, 5, 6]]).T), axis=1)

`np.split` goes in the opposite direction. It will try to produce sub-arrays of equal size. Again there are `hsplit` and `vsplit` variants for common use cases.

In [ ]:
np.split(np.arange(64).reshape((8, 8)), 2, axis=0)

In [ ]:
np.split(np.arange(64).reshape((8, 8)), 2, axis=1)

See also `np.tile`

In [ ]:
np.tile(np.array([[1,0],[0,1]]), (4, 4))

## Universal Functions (ufuncs)

The real reason for using numpy is so you can do numerical operations, _quickly_. Python uses a concept called `ufuncs` or universal function. A ufunc is a function which operates on `ndarrays` element-by-element. More formally, a `ufunc` is a vectorized wrapper around a function which can do a transformation on an `ndarray` and produces another `ndarray`. This element by element behaviour is fundamentally different from the usual python behaviour.

The key to writing fast numeric python code is: **Avoid for & while loops as far as you can, use numpy ufuncs as far as possible**


Lets start with basic arithmetic operations. Numpy can use it's internal broadcasting to do these quickly and efficiently

The usual operations are available

  * +: addition
  * -: subtraction
  * *: multiplication
  * /: division
  * //: integer division
  * **: power operator
  * %: modulo

Remember operations are element by element, and you can build up more complicated expressions as you go

In [ ]:
la = np.linspace(0, 1, 100)

(la ** 2 + la) / (la + 1)

**Example**: Try to calculate the terms of this sum as an `ndarray`
$$
\sqrt{12}\sum_{k=0}^{10}\frac{(-3)^{-k}}{2k+1}
$$

_Hints_: Think term by term. `np.arange(10)` will give you an `ndarray` of k values, next raise `-3. * np.ones(k.shape)` to those powers. If you can calculate the terms you can use the `.sum()` method to sum them all up. How close to $\pi$ did you get?

The operators we were using `+,-,/,...` actually correspond to functions (`ufuncs`)

|operator|function|description|
|--------|--------|-----------|
| + | np.add | Addition |
| - | np.subtract | Subtraction |
| - | np.negative | Unary negation |
| * | np.multiply | Multiplication |
| / | np.division | Ordinary floating point division |
| // | np.floor_divide | floor (integer) division |
| % | np.mod | Modulo/Remainder division |

You can use either syntax, but in the function notation there are lots more functions to play with e.g.

| function | description |
|----------|-------------|
| np.sin   | sin function |
| np.cos   | cos function |
| np.tan   | tan function |
| np.abs   | absolute value |
| np.exp   | exponential |
| np.log   | natural log |
| np.log2  | log base 2 |
| np.log10 | log base 10 |
|  ...     |    ...      |


In [ ]:
p1 = np.linspace(0, 2*np.pi, 25)
p2 = np.sin(p1)
p2

# p2 sin of p1

In [ ]:
q1 = np.linspace(0, 1, 10) + np.linspace(1, 2, 10)*1j
q1

In [ ]:
np.abs(q1)

## Aggregate Functions

Aggregate functions take an `ndarray` and reduce it along one (or more) axes. We used `.sum()` above, but another example would be taking an array of numbers and calculating the mean value...

In [ ]:
r1 = np.linspace(0, 10, 100)
r1.mean()

There are lots of aggregate functions

  * `min`: Minumum value
  * `max`: Maximum value
  * `sum`: Sum values
  * `prod`: Product of values
  * `mean`: Arthmetic mean
  * `std`: Standard deviation
  * `var`: Variance
  * `argmin`: indices of the minimum value
  * `argmax`: indices of the maximum value
  * `all`: is a condition true in all elements
  * `any`: is a condition true in any elements
  
  The default is to reduce along all axes, if you want to reduce along a specific axis you can pass that as an argument (the axes you specify are the ones which get squashed) 

In [ ]:
s1 = np.arange(50)
s2 = s1.reshape(5,10)
s2

In [ ]:
s2.mean(axis=1)

For binary operations (e.g. addition) you can also do reduction, so starting from

[1, 3, 5, 7, 9]

`np.add.reduce` will add 1 to 3, add that to 5 and so on, ...

In [ ]:
t1 = np.arange(1, 10, 2)
np.add.reduce(t1)

N.B. In this case we used a function from the module and passed in our ndarray.

_**Exercise**: Find the value closest to a particular number in an array (try using `np.abs` and `np.argmin`)_

In [122]:
rng = np.random.default_rng(47)
a = rng.normal(size=50)
scalar=1

Numpy also has lots of other utility functions, here are a few commonly used ones

  * `np.bincount`: Count number of occurences of each value in an array of non negative ints
  * `np.allclose`: Check if two arrays are equal within some tolerance
  * `np.pad`: Padding arrays
  * `np.unique`: Find unique elements within an array
  * `np.percentile`: Compute percentiles along some axis
  * `np.sort`: Sort (various algorithms), `np.argsort` returns indices of for sorting
  * `np.where`: Return elements matching some condition
  
and __[many](https://numpy.org/doc/stable/reference/routines.html)__ others.

In [ ]:
np.bincount(np.random.binomial(10, p=0.3, size=50))

## Broadcasting

`numpy` likes to operate element by element, but not all arrays are the same size. To work around this, `numpy` implements a set of rules called `broadcasting` to make `ndarray`s conform whenever possible. This is great news; it means you don't have to worry about doing that yourself, but it is important to understand the rules so that you know how `numpy` will behave when combining differnt shaped `ndarrays`. To get the idea, think of
```python
np.arange(10) * 5
```
`numpy` wants to operate element-by-element, but `5` isn't an `ndarray`, it's just a number. If we could prompte 5 to be a $1\times 5$ `ndarray` with 5's in in every place `numpy` would be happy. This is the basic idea of broadcasting, explicitly

1. Given two arrays of different dimensions, prepend dimensions of length 1 to the shape of the smaller array until they have the same number of dimensions
1. Any dimensions of size 1 are repeated/reused as often as needed to make shapes conform


In [ ]:
a = np.arange(15)
a = a.reshape(3, 5)
a

In [ ]:
b = np.arange(5)
b

If I want to multiply these two `ndarray`s, `b` has fewer dimensions (1 vs. 2) so a dimension of length 1 will be prepended to `b`. `b` will then be repeated 3 times to conform with the shape of a.

In [ ]:
a * b

Just to be explicit, that operation does something lie

In [ ]:
btmp = np.repeat(b[np.newaxis, :], 3, axis=0)
a *  btmp

To take a more complicated example...

In [ ]:
a = np.arange(24).reshape(2,1,3,4)
b = np.arange(8).reshape(2,1,4)

First the number of dimensions are compared; `a` has dimension 4, `b` has dimension 3, so 1 is prepended to the shape of `b`, making it `(1, 2, 1, 4)`. Next we compare the dimensions from right to left. The last dimension matches so nothing is done there. The second to last dimension of `a` is larger then that of `b` so `b` will be repeated 3 times along that axis to make them match. For the next dimension, `b` is larger than `a`, so `a` will be repeated twice, and for the final axis `a` is larger than `b` so `b` will be repeated twice to make them match. Bringing that all together, the final result should have shape `(2,2,3,4)`.

In [ ]:
a+b

In [ ]:
(a+b).shape

## Random

Numpy has a few important submodules but `np.random` is probably the most important. As you might expect, it lets you work with random numbers. It can sample from different distributions (30+ available), handle permutations and do lots of other handy things.

`numpy.random` uses the concept of a `Generators` to implement sampling. The idea is you create a generator object then call methods on that generator to sample from the various distributions. The original `Generator` will normally get it's entropy from a (hopefully reliable source) then you can keep asking it for the `__next__` random elements distributed however you need.

(The generator interface to `numpy.random` is relatively recent, occasionally you will still see me explicity call functions of the submodule as I did above).

In [ ]:
rng = np.random.default_rng(47)

Here we've seeded the `Generator` with a specific value so the results are reporoducible but normally you would just call `np.random.default_rng()` to get a random value from the OS. Now we can sample from various distributions

In [ ]:
rng.normal(5.0, 1.0, (64, 64))

The binomial distribution

In [ ]:
rng.binomial(10, .5, size=20)

5 random integers between 10 and 20 (discrete uniform)

In [ ]:
rng.integers(10, 20, 5)

`np.random` also has functions for shuffling arrays in-place (`np.shuffle`) and selecting elements at random from `ndarrays` (`np.choice`)

In [ ]:
a = np.arange(10)
np.random.shuffle(a)
a

In [ ]:
a = np.arange(10)
np.random.choice(a, 3, replace=False)

You can also weight the selections in `np.choice` with probabilities. e.g. Here are the letter frequencies of ordinary english text.

In [ ]:
letter_freq = {' ': 0.19266420666588144,
 'e': 0.09680968984305797,
 't': 0.07140241019840815,
 'a': 0.06361581392196947,
 'o': 0.06183938572048604,
 'i': 0.05349452953695084,
 'n': 0.0521037201730283,
 'h': 0.051232447485652234,
 's': 0.049280151278754014,
 'r': 0.04524648142979075,
 'd': 0.03375374929612461,
 'l': 0.03124157971419029,
 'u': 0.02392934301198968,
 'm': 0.021518821910249234,
 'w': 0.020208685943305965,
 'c': 0.02004895261728702,
 'f': 0.016262143363080305,
 'y': 0.016250849087503207,
 'g': 0.013559584564274916,
 'p': 0.012933559003715817,
 ',': 0.012259129404969158,
 '.': 0.01200420147051468,
 'b': 0.011160357738111564,
 'v': 0.0076220225466009876,
 'k': 0.006392559976636984,
 'x': 0.001353699601312072,
 'j': 0.0008260955850676769,
 'q': 0.0006663622590487316,
 'z': 0.00031946665203789066
}

We can use those relative frequency values as probabilities (they sum to ~1) and generate a random sample of letters

In [ ]:
letters = np.fromiter(letter_freq.keys(),dtype='<U1')
probabilities = np.fromiter(letter_freq.values(), dtype=float)
chosen = np.random.choice(letters, 1000, p=probabilities)
''.join(chosen)

## See Also

  * [np.linalg](https://numpy.org/doc/stable/reference/routines.linalg.html): Linear algebra operations
  * [np.fft](https://numpy.org/doc/stable/reference/routines.fft.html): Fast Fourier Transforms
  * [Sorting, Searching and Counting](https://numpy.org/doc/stable/reference/routines.sort.html): Sorting

## Exercise

1. Create an `ndarray` `X` random numbers between 0 and 1 (uniformly distributed) with shape `(5, 2)`. We will interprate X as a list of 10 2D coordinates x & y.
2. Use `np.newaxis` and broadcasting to generate a `(5, 5, 2)` array of the squared distances between the each pair of points.
3. Use argsort to build a (10x10) array of nearest neighbours for each point. The values in the array will be integers corresponding to the rows of the original `X`. e.g the first row might look like [0, 3, 2, 1, 4], meaning the nearest point is the point itself (`0` in this case), then the point in the 4th row and so on

## Solution